In [6]:
# A notebook for organising AI progress metrics

from datetime import date

# We have the following structures:
#
# problem 
#     \   \
#      \   metrics  -  measures 
#       \
#        - subproblems
#             \
#           metrics
#              \
#             measures
#
# problems are tagged with attributes:
# eg, vision, abstract-games, language, world-modelling, safety
#     agi       -- most capable humans can do this, so AGIs can do this
#     super     -- the very best humans can do this, or human organisations have solved this
#     verysuper -- neither humans nor human orgs have solved this
#
# problems can have "subproblems", including simpler cases and preconditions
#
# a "metric" is one way of measuring progress on a problem. There will often be several metrics
# for a given problem, but in some cases we'll start out with zero metrics and will need to start
# proposing some...
#
# a measure[ment] is a score on a given metric, by a particular codebase/team/project, at a particular
# time

class Problem:
    def __init__(self, name, attributes=[], solved=False):
        self.name = name
        self.attributes = attributes
        self.subproblems = []
        self.superproblems = []
        self.metrics = []
        self.solved = solved
        
    def subproblem(self, other_problem):
        self.superproblems.append(other_problem)
        other_problem.subproblems.append(self)
        
    def metric(self, *args, **kwargs):
        m = Metric(*args, **kwargs)
        self.metrics.append(m)
        return m


# Different metrics and measurements for progress are made on very different types of scales
# we have some helper functions to regularise these a little bit, so we can tell (for instance)
# whether progress on some metric appears to be accelerating or decelerating.

# Interface:
#    improvement(score1, score2): retrns a consistent measure of how much better score2 is than score1
#    pseudolinear(score): returns a modified version of score where we would expect vaguely linear progress

class Linear:
    def improvement(self, score1, score2):
        return score2 - score1
    def pseudolinear(self, score):
        return score
linear = Linear()

class ELO:
    def improvement(self, score1, score2):
        """
        Normalise an ELO score
        
        An ELO increase of 400 improves your odds by 10x, so we could justify something like
        return 10.0 ** ((score2 - score1)/400.)
        However, it seems that at least for chess ELO progress has been roughly linear over
        time, both for humans and computers (though with different coefficients). Perhaps this
        tracks exponential increases in ability to search the game's state space, driven directly
        by Moore's law on the computer side, and indirectly for humans by access to better training
        tools and more profound libraries of past play.
        
        So for now let's treat this as linear? But ELO is not a chess-specific measure, and in other
        contexts we may want to do exponentiation as documented above?
        """
        return score2 - score1
    def pseudolinear(self, score):
        return score
    
elo = ELO()

class ErrorRate:
    """Many labelling contests use these measures"""
    def improvement(self, score1, score2):
        # 0.5 / 0.25
        return score1 / score2
    def pseudolinear(self, score):
        # The choice of base here is arbitrary. But since this is computer science, let's use base2!
        from math import log
        return log(score) / log(2.0)
error_rate = ErrorRate()
    
class Metric:
    def __init__(self, name, url=None, solved=False, notes="", scale=linear):
        self.name = name
        self.measures = []
        self.solved = solved
        self.url = url
        self.notes = notes
        self.scale = scale
        
    def measure(self, *args, **kwargs):
        m = Measurement(*args, **kwargs)
        self.measures.append(m)
        # Add logic for detection solutions at this point
        return m


class Measurement:
    def __init__(self, date, value, name, url, uncertainty=0, minval=None, maxval=None):
        self.date = date
        self.value = value
        self.name = name
        self.url = url
        self.minval = minval if minval else value - uncertainty
        self.maxval = maxval if maxval else value + uncertainty


### Sources to incorporate

* Jack Clark's collection of progress measurements (elsewhere in this repo, partially imported)
* Sarah Constantin's [Performance Trends in AI](https://srconstantin.wordpress.com/2017/01/28/performance-trends-in-ai/)
* Katja Grace's [Algorithmic Progress in Six Domains](https://intelligence.org/files/AlgorithmicProgress.pdf)

In [7]:
# BEGIN ACTUALLY CLASSIFYING PROBLEMS

scene_description = Problem("Scene description", ["agi", "vision", "language", "world-modelling"])
image_classification = Problem("Image classification", ["vision", "agi"])
scene_description.subproblem(image_classification)

imagenet = image_classification.metric("imagenet", "http://image-net.org", scale=error_rate)
imagenet.notes = """
Correctly label images from the Imagenet dataset. As of 2016, this includes:
 - Object localization for 1000 categories.
 - Object detection for 200 fully labeled categories.
 - Object detection from video for 30 fully labeled categories.
 - Scene classification for 365 scene categories (Joint with MIT Places team) on Places2 Database http://places2.csail.mit.edu.
 - Scene parsing for 150 stuff and discrete object categories (Joint with MIT Places team).
WARNING: these subchallenges were added in successive years of the Imagenet challenge, so results from years are not directly
comparable; however progress should probably be understated by comparing them?
"""

# Data points gathered by Jack Clark:
imagenet.measure(date(2010,8,31), 0.28191, "NEC UIUC", "http://image-net.org/challenges/LSVRC/2010/results")
"""
** 2010: 0.28191**
**NEC UIUC**
http://image-net.org/challenges/LSVRC/2010/results

** 2011: 0.25770
 XRCE**

** 2012: 0.16422**
** Supervision**
http://image-net.org/challenges/LSVRC/2012/results.html

** 2013: 0.11743 **
**Clarifai**
http://www.image-net.org/challenges/LSVRC/2013/results.php

** 2014: 0.07405**
**VGG**
http://image-net.org/challenges/LSVRC/2014/index
 

**2015: 0.03567**
**MSRA**
http://image-net.org/challenges/LSVRC/2015/results

** 2016: 0.02991**
**Trimps-Soushen**
http://image-net.org/challenges/LSVRC/2016/results
* * *
"""
None

In [8]:
# Abstract games like chess, go, checkers etc can be played with no knowldege of the human world
# Although this domain has largely been solved to super-human performance levels, there are a
# few ends that need to be completed in terms of having agents learn rules for arbitrary 
# abstract games effectively

abstract_strategy_games = Problem("Abstract strategy games", ["agi", "abstract-games"])

playing_with_hints = Problem("Playing abstract games with extensive hints", ["abstract-games"], solved=True)
playing_with_hints.notes = """
  Complex abstract strategy games have been solved to super-human levels
  by computer systems with extensive rule-hinting and heuristics,
  in some cases combined with machine learning techniques.
"""
computer_chess = playing_with_hints.metric("computer chess", scale=elo)
# For some caveats, see https://en.wikipedia.org/w/index.php?title=Chess_engine&oldid=764341963#Ratings
computer_chess.measure(date(2017,02,27), 3393, "Stockfish", uncertainty=50,
                           url="https://web.archive.org/web/20170227044521/http://www.computerchess.org.uk/ccrl/4040/")
computer_chess.measure(date(1997,05,11), 2725, "Deep Blue", uncertainty=25,
                           url="https://www.quora.com/What-was-Deep-Blues-Elo-rating")

mastering_historical_games = Problem("Mastering human abstract strategy games", ["super", "abstract-games"])
mastering_chess = mastering_historical_games.metric("mastering chess")
mastering_chess.notes = """
  Beating all humans at chess, given a corpus of past play amongst masters,
  but no human-crafted policy constraints and heuristics. This will probably fall out
  immediately once learning_abstract_game_rules is solved, since playing_with_hints
  has been solved.
"""

# Are there any published metrics for these yet?
learning_abstract_game_rules = Problem("Learning the rules of complex strategy games from examples", ["agi", "abstract-games"])
learning_chess = learning_abstract_game_rules.metric("learning chess")
learning_chess.notes = """
  Chess software contains hard-coded policy constraints for valid play; this metric is whether RL
  or other agents can correctly build those policy constraints from examples or oracles"""
learning_go = learning_abstract_game_rules.metric("learning go")
learning_go.notes = """
  Go software contains policy constraints for valid play and evaluating the number of
  liberties for groups. This metric is whether RL or other agents can correctly build those 
  policy constraints from examples or oracles"""
learning_arbitrary_abstract_games = Problem("Play an arbitrary abstract game, first learning the rules", ["agi", "abstract-games"])
                           


In [9]:
# Speech recognition

"""
        http://melodi.ee.washington.edu/s3tp/

* * *
**_Word error rate on Switchboard (specify details): [Month, Year: Score [SWB]: Team].  Compiled by Jack Clark._**

A note about measurement: We're measuring Switchboard (SWB) and Call Home (CH) performance from the Hub5'00 dataset, with main scores assesses in terms of word error rate on SWB. We also create 

Why do we care: Reflects the improvement of audio processing systems on speech over time.

**2011 16.1%: Microsoft**
16.1% SWB
Who: Microsoft
Technique: Microsoft: CD-DNN (Context Dependent Deep Neural Network)

Paper: Conversational Speech Transcription Using Context-Dependent Neural Networks 
Url: https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/CD-DNN-HMM-SWB-Interspeech2011-Pub.pdf

**2012:**

**April, 2012: 18.5%: University of Toronto, Google, IBM< Microsoft**
Who: University of Toronto, Google, IBM, Microsoft

Paper: Deep Neural Networks for Acoustic Modeling in Speech Recognition
Url: https://pdfs.semanticscholar.org/ce25/00257fda92338ec0a117bea1dbc0381d7c73.pdf?_ga=1.195375081.452266805.1483390947

**2013:**

**2013: 12.9%: Consortium of Universities**
12.9% SWB, 24.5% CH, 18.7% 
Who: Brno University of Tech / University of Edinburgh / Johns Hopkins
Technique: DNN BMMI  (deep neural network boosted minimum mutual information) 

Paper: Sequence discriniative training of deep neural networks 
Url: http://www.danielpovey.com/files/2013_interspeech_dnn.pdf


**2014:**

**June, 2014: 16.0%: Stanford**
16.0% SWB, 23.7% CH, 19.9% EV
Who: Stanford

Paper: Increasing Deep Neural Network Acoustic Model Size For Large Vocabulary Continuous Speech Recognition

Url: https://arxiv.org/abs/1406.7806v1

**December, 2014: 12.6%: Baidu**
Deep Speech SWB: 20.0% SWB, 31.8% CH, 25.9% blended.
Deep Speech SWB + FSH: 12.6% SWB, 19.3% CH, 16.0% blended.

Who: Baidu

Paper: Deep Speech: Scaling up end-to-end speech recognition
Url: https://arxiv.org/abs/1412.5567

**2015:**

**May, 2015: 8%: IBM**
8.0% SWB, 14.1% CH, 11.0% blended.
Who: IBM

Paper: The IBM 2015 English Conversational Telephone Speech Recognition System
Url: https://arxiv.org/abs/1505.05899

**2016: **

**June, 2016: 6.9%: IBM**
n-gram + model M + NNLM: 6.9% SWB, 12.5% CH, 9.7% blended.

Who: IBM
Paper: The IBM 2016 English Conversational Telephone Speech Recognition System

Url: https://arxiv.org/abs/1604.08242v1

**September, 2016: 6.2%: Microsoft**
Single model (ResNet): 6.9% SWB, 13.2% CH, 10.05% blended.
Combination: 6.2% SWB, 12.0% CH, 9.1% blended 

Who: Microsoft
Paper: The Microsoft 2016 Conversational Speech Recognition System
Url: https://arxiv.org/abs/1609.03528

**October 2016: 5.9%: Microsoft**
Single model (ResNet): 6.6% SWB, 12.5% CH, 9.55% blended. 
Combination: 5.9% SWB, 11.1% CH, 8.5% blended.

Who: Microsoft
Paper: Achieving human parity in conversational speech recognition
Url: https://arxiv.org/abs/1610.05256
* * *
"""
None

In [10]:
# Image Generation

""" 
* * *
**_Generative models of CIFAR-10 Natural Images _****[Year: bits-per-subpixel, method]. Compiled by Durk Kingma.**

**Why we care:**
(1) The compression=prediction=understanding=intelligence view (see Hutter prize, etc.). (Note that perplexity, log-likelihood, and #bits are all equivalent measurements.)
(2) Learning a generative model is a prominent auxiliary task towards semi-supervised learning. Current SOTA semi-supervised classification results utilize generative models.
3) You're finding patterns in the data that let you compress it more efficiently. Ultimate pattern recognition benchmark because you're trying to find the patterns in all the data. 

**2014: 4.48**
Method: NICE

Paper: NICE: Non-linear independent components estimation. 
arXiv preprint arXiv:1410.8516, 2014.

**2015: 4.13**
Method: DRAW

Paper: Draw: A recurrent neural network for image generation.
 In International Confer- ence on Machine Learing (ICML), 2015.

**2016: 3.49**
Method: Real NVP

Paper: Density estimation using real NVP. 
arXiv:1605.08803, 2016.

**2016: 3.11**
Method: VAE with IAF

Paper: Improving variational inference with inverse autoregressive flow. 
Conference on Neural Information Processing Systems (NIPS), 2016.

**2016: 3**
Method: PixelRNN

Paper: Density estimation using real NVP. 
arXiv:1605.08803, 2016. arXiv preprint arXiv:1605.08803, 2016.

**2016: 2.92**
Method: PixelCNN++

Paper: PixelCNN++: A PixelCNN implementation with discretized logistic mixture likelihood and other modifications. 
URL: https://openreview.net/forum?id=BJrFC6ceg, 2016.
"""

image_generation = Problem("Generative models of CIFAR-10 Natural Images", ["vision"])
scene_generation = Problem("Be able to generate complex scene e.g. a baboon receiving their degree at convocatoin.", ["vision", "world-modelling"])
scene_generation.subproblem(image_generation)

#image_generation.metric("", "")
